### This is a notebook that walks through the steps to create the temporal data sheet

### Step 1: Import requirements:

In [32]:
import defi.defi_tools as dft
import pandas as pd
import io
import requests
from datetime import datetime, date 
import json



### Step 2: Create a dataframe from llama API with the necessary attributes

In [33]:
##llama API
df = dft.getProtocols()
df['address']=df['address'].str.lower()

## Remove unnecessary* factors
factors_needed=df.filter(['name','address','symbol','audits','category','chains','oracles','tvl','forkedFrom','slug'], axis=1).reset_index()
#Create a new SolaceID
factors_needed['SolaceID'] = factors_needed['name'].str.split(' ').str[0] 
factors_needed['SolaceID']=factors_needed['SolaceID'].str.lower()

# Array with only ethereum protocols
ethProtocols=[]
for i in factors_needed['chains']:
    if 'Ethereum' in i:
        ethProtocols.append(i)

#First 5 values for demonstration
pd.set_option('display.max_rows', None, 'display.max_columns', None)
factors_needed.head()


,name,address,symbol,audits,category,chains,oracles,tvl,forkedFrom,slug,SolaceID
0,Curve,0xd533a949740bb3306d119cc777fa900ba034cd52,CRV,2,Dexes,"[Ethereum, Avalanche, Fantom, Polygon, Arbitru...",[],1.976971e+10,NaN,curve,curve
1,MakerDAO,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,MKR,2,CDP,[Ethereum],[Maker],1.754983e+10,NaN,makerdao,makerdao
2,Convex Finance,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,CVX,2,Yield,[Ethereum],NaN,1.365008e+10,NaN,convex-finance,convex
3,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,Lending,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,NaN,aave,aave
4,WBTC,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,WBTC,2,Bridge,[Ethereum],[],1.160865e+10,NaN,wbtc,wbtc


### Step 3: Get Zapper Api data from this endpoint:

Zapper protocol attributes:

https://api.zapper.fi/v1/apps?api_key=96e0cc51-a62e-42ca-acee-910ea7d2a241

In [34]:

## Attribute Endpoint
url_attributes = r'https://api.zapper.fi/v1/apps?api_key=96e0cc51-a62e-42ca-acee-910ea7d2a241' 
urlData_attributes = requests.get(url_attributes).content
zapperAttributes = pd.read_json(io.StringIO(urlData_attributes.decode('utf-8')))

# Open reference table to create Solace category 
f = open('../governance/reference/mappingTablesSolace.json')
lookupTables = json.loads(f.read())
f.close()
#reference=pd.read_json('../governance/reference/mappingTablesSolace.json')
categories=pd.DataFrame(lookupTables['categorySolaceLookup'])

def lookup_categorySolace(category):
    if pd.isna(category):
        return 'unknown'
    elif category not in list(categories['category']):
        return 'other'
    else:
        return category

zapperAttributes['tags']=zapperAttributes['tags'].str.get(0)
zapperAttributes['categorySolace'] = zapperAttributes['tags'].apply(lookup_categorySolace)

## Merge attrubutes from both llama and zapper using different id from Zapper, Llama and Solace
zapperAttributes['SolaceID'] = zapperAttributes['name'].str.split(' ').str[0]
zapperAttributes['SolaceID']=zapperAttributes['SolaceID'].str.lower()

# First merging on id from zapper and id from llama
combinedTable1=pd.merge(zapperAttributes[['id','tags','name','categorySolace','SolaceID']],factors_needed,left_on='id', right_on='slug', how='left')

combinedTable1=combinedTable1.drop(columns=['SolaceID_y'])
combinedTable1=combinedTable1.rename(columns={'SolaceID_x':'SolaceID'})
# Now merging again on the SolaceID to captue more data points
combinedTable2=pd.merge(combinedTable1,factors_needed,left_on='SolaceID', right_on='SolaceID', how='left')
# Combine both table into one containing all the captured data points
combinedTable2.address_x.fillna(combinedTable2.address_y,inplace=True)

combinedTable2.name_x.fillna(combinedTable2.name_y,inplace=True)

combinedTable2.symbol_x.fillna(combinedTable2.symbol_y,inplace=True)

combinedTable2.chains_x.fillna(combinedTable2.chains_y,inplace=True)

combinedTable2.oracles_x.fillna(combinedTable2.oracles_y,inplace=True)

combinedTable2.tvl_x.fillna(combinedTable2.tvl_y,inplace=True)

combinedTable2.audits_x.fillna(combinedTable2.audits_y,inplace=True)

combinedTable2=combinedTable2.rename(columns={'address_x':'address','name_x':'name','symbol_x':'symbol','chains_x':'chains','oracles_x':'oracles','tvl_x':'tvl','audits_x':'audits'})

combinedTable2.head()


,id,tags,name,categorySolace,SolaceID,name_y,address,symbol,audits,category_x,chains,oracles,tvl,forkedFrom_x,slug_x,name,address_y,symbol_y,audits_y,category_y,chains_y,oracles_y,tvl_y,forkedFrom_y,slug_y
0,aave-amm,lending,Aave AMM,lending,aave,NaN,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,NaN,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,NaN,NaN,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,Lending,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,NaN,aave
1,aave-safety-module,liquidity-pool,Aave Safety Module,liquidity-pool,aave,NaN,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,NaN,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,NaN,NaN,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,Lending,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,NaN,aave
2,aave,lending,Aave,lending,aave,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,Lending,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,NaN,aave,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,Lending,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,NaN,aave
3,aave-v2,lending,Aave V2,lending,aave,NaN,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,NaN,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,NaN,NaN,AAVE,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,Lending,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,NaN,aave
4,aavegotchi,yield-aggregator,Aavegotchi,yield-aggregator,aavegotchi,Aavegotchi,0x3f382dbd960e3a9bbceae22651e88158d2791550,GHST,2,Gaming,"[Polygon, Ethereum]",NaN,1.754204e+07,NaN,aavegotchi,Aavegotchi,0x3f382dbd960e3a9bbceae22651e88158d2791550,GHST,2,Gaming,"[Polygon, Ethereum]",NaN,1.754204e+07,NaN,aavegotchi


### Step 4: Get launch dates from Dune and merge with Table *WIP 

In [35]:
DuneData=pd.read_csv('../temporalDB/launch_dates.csv',names=["address", "LaunchDate"])
finalTable = pd.merge(combinedTable2,DuneData,left_on='address', right_on='address', how='left')
finalTable=finalTable.rename(columns={'id':'appId'})
#Skip these statements if you want all protocols
finalTable=finalTable.loc[finalTable['chains'].isin(ethProtocols)]
finalTable = finalTable.loc[:,~finalTable.columns.duplicated()]
finalTable=finalTable.filter(['appId','tags','name','categorySolace','SolaceID','address','symbol','audits','chains','oracles','tvl','LaunchDate'], axis=1)

finalTable

,appId,tags,name,categorySolace,SolaceID,address,symbol,audits,chains,oracles,tvl,LaunchDate
0,aave-amm,lending,Aave AMM,lending,aave,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,2020-09-24 18:06
1,aave-safety-module,liquidity-pool,Aave Safety Module,liquidity-pool,aave,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,2020-09-24 18:06
2,aave,lending,Aave,lending,aave,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,2020-09-24 18:06
3,aave-v2,lending,Aave V2,lending,aave,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,2020-09-24 18:06
4,aavegotchi,yield-aggregator,Aavegotchi,yield-aggregator,aavegotchi,0x3f382dbd960e3a9bbceae22651e88158d2791550,GHST,2,"[Polygon, Ethereum]",NaN,1.754204e+07,NaN
5,abracadabra,lending,Abracadabra,lending,abracadabra,0x090185f2135308bad17527004364ebcc2d37e5f6,SPELL,2,"[Ethereum, Arbitrum, Fantom, Avalanche]",[Chainlink],3.130727e+09,2021-05-17 22:10
7,alchemix,lending,Alchemix,lending,alchemix,0xdbdb4d16eda451d0503b854cf79d55697f90c8df,ALCX,2,[Ethereum],[],9.604898e+08,2021-02-27 04:28
8,alkemi,lending,Alkemi Network,lending,alkemi,0x6c16119b20fa52600230f074b349da3cb861a7e3,ALK,2,[Ethereum],[Chainlink],9.860368e+06,2021-09-02 17:35
9,alpha-tokenomics,elastic-finance,Alpha Tokenomics,other,alpha,0xa1faa113cbe53436df28ff0aee54275c13b40975,ALPHA,2,"[Avalanche, Ethereum, Binance]","[Band, Chainlink]",8.278031e+08,2020-09-27 09:10
10,alpha-v1,lending,Alpha,lending,alpha,0xa1faa113cbe53436df28ff0aee54275c13b40975,ALPHA,2,"[Avalanche, Ethereum, Binance]","[Band, Chainlink]",8.278031e+08,2020-09-27 09:10


### Step 5: Create the current age column 

In [36]:
#Current Date
now = pd.Timestamp('now')
finalTable['LaunchDate'] = pd.to_datetime(finalTable['LaunchDate'], format="%Y-%m-%d %H:%M")    
finalTable['age'] = (now - finalTable['LaunchDate']).astype('<m8[M]')    #Age in months
finalTable = finalTable.drop_duplicates(subset=['appId'], keep='first')
finalTable

,appId,tags,name,categorySolace,SolaceID,address,symbol,audits,chains,oracles,tvl,LaunchDate,age
0,aave-amm,lending,Aave AMM,lending,aave,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,2020-09-24 18:06:00,16.0
1,aave-safety-module,liquidity-pool,Aave Safety Module,liquidity-pool,aave,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,2020-09-24 18:06:00,16.0
2,aave,lending,Aave,lending,aave,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,2020-09-24 18:06:00,16.0
3,aave-v2,lending,Aave V2,lending,aave,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,AAVE,2,"[Ethereum, Avalanche, Polygon]",[Chainlink],1.359565e+10,2020-09-24 18:06:00,16.0
4,aavegotchi,yield-aggregator,Aavegotchi,yield-aggregator,aavegotchi,0x3f382dbd960e3a9bbceae22651e88158d2791550,GHST,2,"[Polygon, Ethereum]",NaN,1.754204e+07,NaT,NaN
5,abracadabra,lending,Abracadabra,lending,abracadabra,0x090185f2135308bad17527004364ebcc2d37e5f6,SPELL,2,"[Ethereum, Arbitrum, Fantom, Avalanche]",[Chainlink],3.130727e+09,2021-05-17 22:10:00,8.0
7,alchemix,lending,Alchemix,lending,alchemix,0xdbdb4d16eda451d0503b854cf79d55697f90c8df,ALCX,2,[Ethereum],[],9.604898e+08,2021-02-27 04:28:00,11.0
8,alkemi,lending,Alkemi Network,lending,alkemi,0x6c16119b20fa52600230f074b349da3cb861a7e3,ALK,2,[Ethereum],[Chainlink],9.860368e+06,2021-09-02 17:35:00,5.0
9,alpha-tokenomics,elastic-finance,Alpha Tokenomics,other,alpha,0xa1faa113cbe53436df28ff0aee54275c13b40975,ALPHA,2,"[Avalanche, Ethereum, Binance]","[Band, Chainlink]",8.278031e+08,2020-09-27 09:10:00,16.0
10,alpha-v1,lending,Alpha,lending,alpha,0xa1faa113cbe53436df28ff0aee54275c13b40975,ALPHA,2,"[Avalanche, Ethereum, Binance]","[Band, Chainlink]",8.278031e+08,2020-09-27 09:10:00,16.0


### Step 6: Creating excel table and formatting

In [38]:


writer = pd.ExcelWriter('../temporalDB/TemporalDataset.xlsx', engine='xlsxwriter')

finalTable.to_excel(writer, sheet_name='TemporalData', startrow=1, header=False, index=False)

workbook = writer.book
worksheet = writer.sheets['TemporalData']

(max_row, max_col) = finalTable.shape

#Column Headers
column_settings = []
for header in finalTable.columns:
    column_settings.append({'header': header})

# Add the table.
worksheet.add_table(0, 0, max_row, max_col - 1, {'columns': column_settings})

# Make the columns wider for clarity.
worksheet.set_column(0, max_col - 1, 12)
worksheet.set_column('B:B', 40)
worksheet.set_column('E:E', 40)
worksheet.set_column('H:H', 40)
worksheet.set_column('K:K', 40)
worksheet.set_column('I:I', 40)
worksheet.set_column('C:C', 40)
worksheet.set_column('D:D', 40)


writer.save()


### Step 7: Deploy to AWS *WIP

### Step 8: Automatic updates *WIP